In [3]:
import math
import numpy as np
import pandas as pd
import scipy.stats as spst  
import scipy.special as spsp

from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.tools.tools import add_constant
from statsmodels.formula.api import poisson, negativebinomial

import warnings
warnings.filterwarnings('ignore')

# Setup

In this problem, we will be working with a data set that tracks the the number of bike crossings over all bridges between the East River and Manhattan on different dates. We also have data on weather, specifically temperature and precipitation. This information may be useful for making decisions on when to schedule bike lane closures for maintenance, for example.

The relevant columns are as follows:

* `Total` (outcome): The number of total bike crossings across the east river bridges (Manhattan Bridge, Brooklyn Bridge, Williamsburg Bridge, and Queensboro Bridge).

* `HighTemp`: The high temperature for a specific date.

* `LowTemp`: The low temperature for a specific date.

* `Precipitation`: Precipitation levels for a specific date.

Reference to the raw dataset [here](https://www.kaggle.com/datasets/new-york-city/nyc-east-river-bicycle-crossings).

In [37]:
data = pd.read_csv('nyc-east-river-bicycle-counts.csv')
data = data[['Date', 
             'HighTemp', 
             'LowTemp', 
             'Precipitation', 
             'Total']]
data.head()

,Date,HighTemp,LowTemp,Precipitation,Total
0,2016-04-01 00:00:00,78.1,66.0,0.01,11497
1,2016-04-02 00:00:00,55.0,48.9,0.15,6922
2,2016-04-03 00:00:00,39.9,34.0,0.09,4759
3,2016-04-04 00:00:00,44.1,33.1,0.47,4335
4,2016-04-05 00:00:00,42.1,26.1,0.00,9471


# Part 1 [12 pts]

* Run a Poisson regression using `poisson()` from `statsmodels` to predict `Total` from the other three variables. Show the model summary.

* Give an intuitive explanation for each of the four estimated coefficients. Are they reliable and statistically significant estimates?

* Use the model to predict the average number of bike crossings under two scenarios: a dry winter day with temperature range $25^\circ$ to $35^\circ$ and $0.1$ precipitation level, and a wet summer day with temperature range $75^\circ$ to $80^\circ$ and $0.8$ precipitation level.



In [36]:
result = poisson("Total~HighTemp+LowTemp+Precipitation", data).fit()
result.summary()

Optimization terminated successfully.
         Current function value: 362.376170
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:                  Total   No. Observations:                  210
Model:                        Poisson   Df Residuals:                      206
Method:                           MLE   Df Model:                            3
Date:                Tue, 24 Sep 2024   Pseudo R-squ.:                  0.6913
Time:                        15:36:34   Log-Likelihood:                -76099.
converged:                       True   LL-Null:                   -2.4651e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         8.3964      0.004   2265.587      0.000       8.389       8.404
HighTemp          0.0244   9.65e-05    253.205      0.000       0.024       0.025
LowTemp          -0.0052      0.000    -48.165      0.000      -0.005      -0.005
Precipitation    -2.2184      0.009   -248.186      0.000      -2.236      -2.201
=================================================================================
"""

# part 2
Yes, they are statistical significant because we see they all with p value smaller than 0.05. They all also have std err smaller than coef. They are also reliable as we can see the conficence interval of them are small.

In [38]:
s1 = pd.DataFrame({
    'HighTemp': [35],
    'LowTemp': [25],
    'Precipitation': [0.1]
})

s2 = pd.DataFrame({
    'HighTemp': [80],
    'LowTemp': [75],
    'Precipitation': [0.8]
})

pred1 = result.predict(s1)
pred1

0    7336.368443
dtype: float64

In [39]:
pred2 = result.predict(s2)
pred2

0    3601.82271
dtype: float64

# Part 2 [4 pts]

Suppose we are interested in computing the log-likelihood of a Poisson mixture model, given as 
$$\log \mathcal L(y_i|\lambda_1,\lambda_2) = \log (f(y_i|\lambda_1)p + f(y_i|\lambda_2)(1-p))$$
where $f$ is the PMF of the Poisson distribution. 
* Try to compute the joint log-likelihood $\log \mathcal L(\mathbf y | \lambda_1, \lambda_2)$ of the synthetic data provided below using the Poisson mixture model $\lambda_1=2000, \lambda_2=8000, p=0.7$. Use the functions `spst.poisson.pmf()` and `np.log()`; what is the result?

An alternative method utilizes the [LogSumExp](https://en.wikipedia.org/wiki/LogSumExp) function, a smooth approximation of the max function: 
$$LSE(x_1, ..., x_n) = \log(\exp(x_1)+\cdots+\exp(x_n))$$ 
Notice that while $\log \mathcal L$ is not exacty in the $LSE$ form, you can apply the transformation $x = \exp(\log x)$ to the summands. 
* Using this idea, compute the joint log-likelihood using the same data again. This time, use the functions `spst.poisson.logpmf()` and `spsp.logsumexp()`.

In [40]:
y = np.array([4472, 4457, 4494, 4597, 4514, 4511, 4448, 4507, 4553, 4518, 4443,
              4354, 4456, 4476, 4414, 4471, 4500, 4554, 4520, 4585, 4417, 4569,
              4551, 4531, 4590, 4469, 4448, 4428, 4546, 4462, 4561, 4512, 4413,
              4517, 4486, 4478, 4622, 4613, 4423, 4461, 4593, 4474, 4430, 4420,
              4526, 4497, 4506, 4483, 4441, 4495, 4552, 4457, 4530, 4622, 4496,
              4549, 4554, 4477, 4487, 4581, 4601, 4454, 4561, 4469, 4567, 4364,
              4529, 4413, 4530, 4497, 4548, 4506, 4546, 4480, 5396, 5491, 5612, 
              5388, 5527, 5695, 5526, 5510, 5588, 5430, 5483, 5500, 5407, 5542, 
              5546, 5375, 5425, 5496, 5460, 5517, 5516, 5557, 5622, 5541, 5476, 5523])

In [41]:
lambda1 = 2000
lambda2 = 8000
p = 0.7

def LLS_comp (y, lambda1, lamda2, p):
    LLs = 0
    for yi in y:
        LLs = LLs + np.log(spst.poisson.pmf(yi, lambda1) * p + spst.poisson.pmf(yi, lambda2) * (1 - p))
    return LLs

def log_sum_exp (y, lambda1, lamda2, p):
    LLs = 0
    for yi in y:
        LLs = LLs + spsp.logsumexp([spst.poisson.logpmf(yi, lambda1) + np.log(p), 
        spst.poisson.logpmf(yi, lambda2) + np.log(1 - p)])
    return LLs

log1 = LLS_comp(y, lambda1, lambda2, p)
log2 = log_sum_exp(y, lambda1, lambda2, p)

print(log1, log2)


-inf -79343.02271756018


# Part 3 [12 pts]

Define a new `GenericLikelihoodModel` class and the associated `loglike()` method in order to run a 2-segment Poisson regression with covariates on the bicycle data set. Let each of $\lambda_1$ and $\lambda_2$ depend on `HighTemp`, `LowTemp`, and `Precipitation`. Including the $p$ parameter, that gives us **nine** parameters in total to estimate. 

In your `loglike()` method, remember that you will want to estimate the logistic transformation of $p$. Also, you will want to use `logsumexp()` as in Part 2 above in order to compute log likelihoods. Otherwise, your estimate will very likely not converge.

To perform the actual estimation, use the `bfgs` method, and initialize all parameters to $0$. Show the model summary when done.

In [42]:
class PoissonMix(GenericLikelihoodModel):
    def loglike(self, params):
        X, y = self.exog, self.endog
        lambda1 = np.exp(X @ params[0:4])
        lambda2 = np.exp(X @ params[4:8])
        p = np.exp(params[8]) / (1+np.exp(params[8]))
        ll = spsp.logsumexp([spst.poisson.logpmf(y, lambda1) + np.log(p), 
        spst.poisson.logpmf(y, lambda2) + np.log(1 - p)], axis = 0)
        return np.sum(ll)

In [43]:
X = data[['HighTemp','LowTemp','Precipitation']]
X = add_constant(X)
y = data['Total']
result = PoissonMix(y, X).fit(start_params=np.zeros(9), method='bfgs')
result.summary(xname=['b10','b11','b12','b20','b21','b22','b30','b31','b32'])

         Current function value: 108.959095
         Iterations: 49
         Function evaluations: 152
         Gradient evaluations: 139


<class 'statsmodels.iolib.summary.Summary'>
"""
                              PoissonMix Results                              
==============================================================================
Dep. Variable:                  Total   Log-Likelihood:                -22881.
Model:                     PoissonMix   AIC:                         4.578e+04
Method:            Maximum Likelihood   BIC:                         4.581e+04
Date:                Tue, 24 Sep 2024                                         
Time:                        15:37:14                                         
No. Observations:                 210                                         
Df Residuals:                     206                                         
Df Model:                           3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
b10            8.2670      0.006   1390.874      0.000       8.255       8.279
b11            0.0225      0.000    128.356      0.000       0.022       0.023
b12            0.0034      0.000     19.666      0.000       0.003       0.004
b20           -5.2254      0.028   -186.615      0.000      -5.280      -5.171
b21            7.5725      0.006   1171.518      0.000       7.560       7.585
b22            0.0455      0.000    307.404      0.000       0.045       0.046
b30           -0.0223      0.000   -140.899      0.000      -0.023      -0.022
b31           -0.3472      0.010    -33.282      0.000      -0.368      -0.327
b32            0.1335      0.138      0.965      0.334      -0.138       0.405
==============================================================================
"""

# Part 4 [6 pts]

Consider a lovely day with with temperature range $65^\circ$ to $75^\circ$ and $0.1$ precipitation level. Use your learned model to answer the following:

* What is the expected number of total bike crossings given that the day belongs in segment 1? What if it belongs in segment 2?

* What is the probability that a generic day belongs to segment 1? Use the full mixture model to predict the expected number of total bike crossings for the given day.

In [44]:
x = ([1, 75, 65, 0.1]) # intercept, HighTemp, LowTemp, P
lambda1 = np.exp(x @ result.params[0:4])
lambda2 = np.exp(x @ result.params[4:8])
p = np.exp(result.params[8]) / (1 + np.exp(result.params[8]))

print(lambda1, lambda2, p, lambda1 * p + lambda2 * (1 - p))

15483.724657824314 13447.806954614025 0.5333334075846211 14533.629880829023
